### Diferenças entre AdaBoosting e GBM (Gradient Boosting Machine)

1. **Base Learners**:
   - *AdaBoosting*: Usa um único modelo fraco por vez (geralmente árvores de decisão rasas) e ajusta os pesos das observações incorretas em cada iteração.
   - *GBM*: Usa múltiplos modelos fracos que corrigem os erros residuais dos modelos anteriores através da soma dos gradientes.

2. **Peso das Amostras vs. Erros Residuais**:
   - *AdaBoosting*: Foca em ajustar o peso das amostras classificadas incorretamente em cada iteração, dando mais atenção a estas amostras no próximo modelo.
   - *GBM*: Ajusta os erros residuais diretamente, minimizando uma função de perda diferenciável (como erro quadrático ou log-loss).

3. **Função de Perda**:
   - *AdaBoosting*: Trabalha implicitamente para minimizar a exponencial da perda de classificação.
   - *GBM*: Oferece flexibilidade para usar diferentes funções de perda, dependendo do problema (regressão ou classificação).

4. **Método de Atualização**:
   - *AdaBoosting*: Calcula pesos para cada modelo fraco com base em seu desempenho global.
   - *GBM*: Constrói modelos sucessivos para reduzir os gradientes dos erros residuais do modelo anterior.

5. **Velocidade e Complexidade**:
   - *AdaBoosting*: Geralmente mais rápido, pois cada modelo se concentra em pesos ajustados, sem a necessidade de calcular gradientes complexos.
   - *GBM*: Tende a ser mais lento, pois a otimização de gradientes exige cálculos mais intensos.

### Exemplo de Uso
- *AdaBoosting*: Melhor para datasets menores e mais simples onde o foco é classificação rápida.
- *GBM*: Mais adequado para problemas complexos, onde o ajuste fino do modelo com gradientes pode trazer maior precisão.


### Exemplo Classificação

In [26]:
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier

X, y = make_hastie_10_2(random_state=0)
X_train, X_test = X[:2000], X[2000:]
y_train, y_test = y[:2000], y[2000:]

clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
    max_depth=1, random_state=0).fit(X_train, y_train)
clf.score(X_test, y_test)

0.913

### Exemplo de Regressão

In [24]:
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_friedman1
from sklearn.ensemble import GradientBoostingRegressor

X, y = make_friedman1(n_samples=1200, random_state=0, noise=1.0)
X_train, X_test = X[:200], X[200:]
y_train, y_test = y[:200], y[200:]
est = GradientBoostingRegressor(
    n_estimators=100, learning_rate=0.1, max_depth=1, random_state=0,
    loss='squared_error'
).fit(X_train, y_train)
mean_squared_error(y_test, est.predict(X_test))

np.float64(5.009154859960321)

### 5 Hiperparâmetros Importantes no GBM (Gradient Boosting Machine)

1. **`n_estimators`**:
   - Representa o número de árvores (ou etapas de boosting) a serem treinadas.
   - Valores maiores podem aumentar a capacidade do modelo, mas também aumentam o risco de overfitting.
   - Exemplo: `n_estimators=100`

2. **`learning_rate`**:
   - Taxa de aprendizado que controla o impacto de cada árvore adicionada.
   - Valores menores (ex.: `0.01` ou `0.1`) exigem mais árvores para alcançar uma boa performance, mas geralmente resultam em melhor generalização.
   - Exemplo: `learning_rate=0.1`

3. **`max_depth`**:
   - Profundidade máxima de cada árvore individual.
   - Controla a complexidade do modelo e previne overfitting em datasets pequenos.
   - Exemplo: `max_depth=3`

4. **`min_samples_split`**:
   - Número mínimo de amostras necessárias para dividir um nó.
   - Valores maiores levam a árvores mais simples e podem ajudar a evitar overfitting.
   - Exemplo: `min_samples_split=2`

5. **`subsample`**:
   - Proporção de amostras utilizadas para treinar cada árvore.
   - Um valor menor que 1 (ex.: `subsample=0.8`) introduz aleatoriedade e pode melhorar a generalização do modelo.
   - Exemplo: `subsample=0.8`

### Observação
A escolha e o ajuste desses hiperparâmetros devem ser feitos com cuidado, normalmente usando técnicas como validação cruzada ou otimização automatizada (ex.: Grid Search ou Random Search).


### Melhores Parâmetros Classificação

In [33]:
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Gerar os dados
X, y = make_hastie_10_2(random_state=0)
X_train, X_test = X[:2000], X[2000:]
y_train, y_test = y[:2000], y[2000:]

# Definir o modelo
model = GradientBoostingClassifier(random_state=0)

# Definir os hiperparâmetros a serem testados
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 1.0],
    'max_depth': [1, 2, 3],
    'subsample': [0.8, 1.0],
    'loss': ['log_loss', 'exponential']  # Corrigir o erro nos valores do parâmetro "loss"
}

# Configurar o GridSearchCV
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,  # Validação cruzada com 5 folds
    n_jobs=-1,  # Usar todos os núcleos disponíveis
    verbose=1
)

# Treinar o GridSearch
grid_search.fit(X_train, y_train)

# Resultados
print("Melhores parâmetros encontrados:", grid_search.best_params_)
print("Melhor precisão (accuracy):", grid_search.best_score_)

# Avaliação no conjunto de teste
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print("Precisão no conjunto de teste:", test_accuracy)


Fitting 5 folds for each of 108 candidates, totalling 540 fits
Melhores parâmetros encontrados: {'learning_rate': 1.0, 'loss': 'log_loss', 'max_depth': 1, 'n_estimators': 150, 'subsample': 1.0}
Melhor precisão (accuracy): 0.9279999999999999
Precisão no conjunto de teste: 0.9279


### Melhores Parâmetros Regressão

In [36]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_friedman1
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

# Gerar os dados
X, y = make_friedman1(n_samples=1200, random_state=0, noise=1.0)
X_train, X_test = X[:200], X[200:]
y_train, y_test = y[:200], y[200:]

# Definir o modelo
model = GradientBoostingRegressor(random_state=0)

# Definir os hiperparâmetros a serem testados
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [1, 2, 3],
    'subsample': [0.8, 1.0],
    'loss': ['squared_error', 'absolute_error']
}

# Configurar o GridSearchCV
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=5,
    n_jobs=-1,
    verbose=1,
    error_score='raise'
)

# Treinar o GridSearch
grid_search.fit(X_train, y_train)

# Resultados
print("Melhores parâmetros encontrados:", grid_search.best_params_)
print("Melhor MSE (negativo):", grid_search.best_score_)

# Avaliação no conjunto de teste
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
test_mse = mean_squared_error(y_test, y_pred)
print("MSE no conjunto de teste:", test_mse)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Melhores parâmetros encontrados: {'learning_rate': 0.2, 'loss': 'absolute_error', 'max_depth': 2, 'n_estimators': 150, 'subsample': 0.8}
Melhor MSE (negativo): -5.200984221648046
MSE no conjunto de teste: 3.3384694218420607


### Diferença entre Gradient Boosting Machine (GBM) e Stochastic GBM

A maior diferença entre o **Stochastic Gradient Boosting Machine (Stochastic GBM)** e o **Gradient Boosting Machine (GBM)** tradicional está na forma como eles utilizam os dados durante o treinamento de cada árvore.

#### Stochastic GBM
No Stochastic GBM, uma **amostragem aleatória** (subconjunto) dos dados de treinamento é usada para ajustar cada árvore. Essa amostragem é controlada pelo hiperparâmetro **`subsample`**, que especifica a proporção dos dados a serem usados em cada iteração:

- `subsample=1.0`: Comportamento idêntico ao GBM tradicional (todos os dados são usados).
- `subsample<1.0`: Apenas uma fração dos dados é usada para treinar cada árvore, introduzindo aleatoriedade.

#### Impactos da Diferença

1. **Redução do Overfitting**:
   - No Stochastic GBM, a introdução da aleatoriedade ajuda a reduzir o risco de overfitting, especialmente em datasets pequenos ou com ruído.
   - O GBM tradicional usa todo o conjunto de treinamento, o que pode levar a um ajuste excessivo.

2. **Maior Robustez**:
   - O uso de amostragem aleatória no Stochastic GBM torna o modelo mais robusto a pequenas variações nos dados, melhorando a generalização.

3. **Eficiência Computacional**:
   - Como o Stochastic GBM treina cada árvore em um subconjunto dos dados, ele pode ser mais rápido, especialmente em datasets grandes.

#### Resumo
A introdução da amostragem aleatória no Stochastic GBM é uma ideia inspirada no *bagging* (como no Random Forest), mas aplicada dentro do contexto do boosting. Isso o torna mais eficiente e menos propenso ao overfitting do que o GBM tradicional.
